In [30]:
#Imported all the files 
import cv2
from skimage import data, io, filters
from PIL import Image
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [31]:
# Imported All the files 
train = pd.read_csv('D:\Kaggle Projects\Digit Recognizer\\train.csv')
test = pd.read_csv('D:\Kaggle Projects\Digit Recognizer\\test.csv')
submission = pd.read_csv('D:\Kaggle Projects\Digit Recognizer\\sample_submission.csv')

In [32]:
# taking the Data of train head in the form of array 
train_head = train.iloc[:,0].values.ravel()

In [33]:
train_head

array([1, 0, 1, ..., 7, 6, 9], dtype=int64)

In [34]:
#dropping the label from the train Data 
train_new = train.drop('label', 1)

In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [36]:
#taking the pca as 48 coz less data showing more variance !! 
pca = PCA(n_components=48, whiten=True)

In [37]:
pca.fit(train_new)

PCA(copy=True, n_components=48, whiten=True)

In [38]:
var= pca.explained_variance_ratio_

In [39]:
train_new.head(2)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#Checking the Cummulative variance of the Data (Hit and try for several times to get the best variance)
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

In [41]:
print var1

[  9.75  16.91  23.06  28.44  33.33  37.63  40.91  43.8   46.57  48.92
  51.02  53.08  54.78  56.47  58.05  59.53  60.85  62.13  63.32  64.47
  65.54  66.56  67.52  68.43  69.32  70.16  70.97  71.75  72.49  73.18
  73.84  74.48  75.08  75.67  76.23  76.77  77.28  77.77  78.25  78.72
  79.17  79.61  80.03  80.43  80.81  81.18  81.54  81.89]


In [42]:
# transform the train data into PCA data (variance Data )
traindata_pca = pca.transform(train_new)


In [43]:
#Transforming the test Data to variance PCA data
testdata_pca = pca.transform(test)

In [44]:
traindata_pca

array([[ 1.14345153,  1.410294  , -0.39896126, ...,  0.29444235,
        -0.48605666,  0.51458932],
       [-2.94065892,  0.72712063,  1.09231102, ..., -0.37978246,
        -0.36532851,  1.38223026],
       [ 1.53284049,  0.59243445, -0.14618108, ...,  1.18106799,
         0.06104417,  0.10291324],
       ..., 
       [-0.58118331, -1.9158861 ,  0.36701375, ..., -0.0509583 ,
         0.14636248,  0.04958198],
       [-0.56585206, -0.12506721, -1.51837028, ..., -0.42429361,
        -0.01753014, -0.53483306],
       [ 0.62954045, -0.84477728, -0.1369179 , ...,  0.19783571,
         0.1358208 ,  0.48220483]])

In [45]:
#Putting the Support vector machine on PCA Data 
svc = SVC(C=100,tol=0.0008)



In [46]:
# fitting the Model on the same 
svc.fit(traindata_pca,train_head)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.0008, verbose=False)

In [47]:
#Predicting the values based on the Data 
pred = svc.predict(testdata_pca)

In [48]:
pred

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [49]:
#Below Steps Creating the Submission file 
pred_new = pd.DataFrame(pred)
pred_new.rename(columns={0 : 'Label'}, inplace=True)
submission = submission.drop('Label', 1)
bigdata = pd.concat([submission, pred_new],  axis=1)
bigdata.to_csv('D:\Kaggle Projects\Digit Recognizer\\Rats001.csv' ,index=False)